In [1]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._

Intitializing Scala interpreter ...

Spark Web UI available at http://172.16.18.84:4040
SparkContext available as 'sc' (version = 3.5.0, master = local[*], app id = local-1701225186776)
SparkSession available as 'spark'


import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._


In [2]:
val spark = SparkSession
    .builder
    .appName("ch5")
    .enableHiveSupport()
    .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3578f63e


In [3]:
val cubed = (s: Long) => {
    s * s * s
}

cubed: Long => Long = $Lambda$2046/1979253515@62b41df


In [4]:
spark.udf.register("cubed", cubed)

res0: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$2046/1979253515@62b41df,LongType,List(Some(class[value[0]: bigint])),Some(class[value[0]: bigint]),Some(cubed),false,true)


In [6]:
spark.range(1, 9).createOrReplaceTempView("udf_test")

In [7]:
spark.sql("""
select
    id,
    cubed(id) as id_cubed
    from udf_test
""").show()

+---+--------+
| id|id_cubed|
+---+--------+
|  1|       1|
|  2|       8|
|  3|      27|
|  4|      64|
|  5|     125|
|  6|     216|
|  7|     343|
|  8|     512|
+---+--------+



In [8]:
val t1 = Array(35, 36, 32, 30, 40, 42, 38)
val t2 = Array(31, 32, 34, 55, 56)
val tc = Seq(t1, t2).toDF("celsius")

t1: Array[Int] = Array(35, 36, 32, 30, 40, 42, 38)
t2: Array[Int] = Array(31, 32, 34, 55, 56)
tc: org.apache.spark.sql.DataFrame = [celsius: array<int>]


In [9]:
tc.createOrReplaceTempView("tc")

In [10]:
tc.show()

+--------------------+
|             celsius|
+--------------------+
|[35, 36, 32, 30, ...|
|[31, 32, 34, 55, 56]|
+--------------------+



In [12]:
spark.sql(
"""
select
    celsius,
    transform(celsius, t -> ((t*9) div 5) + 32) as fahrenheit
from tc
"""
).show()

+--------------------+--------------------+
|             celsius|          fahrenheit|
+--------------------+--------------------+
|[35, 36, 32, 30, ...|[95, 96, 89, 86, ...|
|[31, 32, 34, 55, 56]|[87, 89, 93, 131,...|
+--------------------+--------------------+



In [13]:
spark.sql(
"""
select
     celsius,
     filter(celsius, t -> t > 38) as high
from tc
"""
).show()

+--------------------+--------+
|             celsius|    high|
+--------------------+--------+
|[35, 36, 32, 30, ...|[40, 42]|
|[31, 32, 34, 55, 56]|[55, 56]|
+--------------------+--------+



In [14]:
spark.sql(
"""
select
    celsius,
    exists(celsius, t -> t = 38) as threshold
from tc
"""
).show()

+--------------------+---------+
|             celsius|threshold|
+--------------------+---------+
|[35, 36, 32, 30, ...|     true|
|[31, 32, 34, 55, 56]|    false|
+--------------------+---------+



In [16]:
spark.sql(
"""
select
    celsius,
    reduce(celsius, 0, (t, acc) -> acc + t, acc -> (acc div size(celsius) * 9 div 5) + 32) as avgFahrenheit
from tc
"""
).show()

+--------------------+-------------+
|             celsius|avgFahrenheit|
+--------------------+-------------+
|[35, 36, 32, 30, ...|           96|
|[31, 32, 34, 55, 56]|          105|
+--------------------+-------------+



In [3]:
val delay_path = "./data/departuredelays.csv"

delay_path: String = ./data/departuredelays.csv


In [4]:
val airport_path = "./data/airport-codes-na.txt"

airport_path: String = ./data/airport-codes-na.txt


In [5]:
val airport = spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("delimiter", "\t")
    .csv(airport_path)

airport: org.apache.spark.sql.DataFrame = [City: string, State: string ... 2 more fields]


In [7]:
airport.createOrReplaceTempView("airports_na")

In [8]:
val delay = spark.read
    .option("header", "true")
    .csv(delay_path)
    .withColumn("delay", expr("CAST(delay as INT) as delay"))
    .withColumn("distance", expr("CAST(distance as int) as distance"))

delay: org.apache.spark.sql.DataFrame = [date: string, delay: int ... 3 more fields]


In [9]:
delay.createOrReplaceTempView("departureDelays")

In [10]:
val foo = delay.filter(
    expr("""origin == 'SEA' AND destination == 'SFO'
    AND date like '01010%' AND delay > 0 """)
)

foo: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [date: string, delay: int ... 3 more fields]


In [12]:
foo.createOrReplaceTempView("foo")

In [13]:
spark.sql(
    """
    select 
    *
    from airports_na
    limit 10
    """
).show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
+-----------+-----+-------+----+



In [14]:
spark.sql(
    """
    select 
    *
    from departureDelays
    limit 10
    """
).show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+



In [15]:
spark.sql(
    """
    select 
    *
    from foo
    limit 10
    """
).show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [17]:
val bar = delay.union(foo)

bar: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [date: string, delay: int ... 3 more fields]


In [18]:
bar.createOrReplaceTempView("bar")

In [19]:
bar.filter(expr("""origin == 'SEA' AND destination == 'SFO' 
AND date like '01010%' AND delay > 0
""")).show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [20]:
foo.join(
    airport.as("air"),
    $"air.IATA" === $"origin"
).select("City","State", "date", "delay", "distance", "destination").show()

+-------+-----+--------+-----+--------+-----------+
|   City|State|    date|delay|distance|destination|
+-------+-----+--------+-----+--------+-----------+
|Seattle|   WA|01010710|   31|     590|        SFO|
|Seattle|   WA|01010955|  104|     590|        SFO|
|Seattle|   WA|01010730|    5|     590|        SFO|
+-------+-----+--------+-----+--------+-----------+

